# 심층 신경망 학습 시키기
고해상도 이미지에서 수백 종류의 물체를 감지하는 것처럼 아주 복잡한 문제를 다루기 위해선 훨씬 더 넓고 깊은 신경망을 학습해야 함.  
신경망의 깊이가 깊어질수록 다음과 같은 문제가 발생할 수 있음.  
- **그래디언트 소실, 폭주** 문제에 직면할 수 있음. 심층 신경망의 아래쪽으로(입력쪽) 갈수록 그래디언트가 점점 더 작아지거나 커지는 현상을 말함. 두 현상 모두 하위층을 학습시키기 어렵게 만듦.  
- 학습 데이터가 충분하지 않거나 레이블 작업에 비용이 너무 많이 들 수 있음.  
- 학습이 매우 느려질 수 있음  
- **과대 적합** 의 위험이 매우 큼. 특히 학습 샘플이 적거나 잡음이 많은 경우에 심함  
  
이 장에선 해당 문제를 차례로 살펴보고 해결 방법을 제시해줌.  

---
# 1. 그래디언트 소실, 폭주
- 역전파 알고리즘이 출력층에서 입력층으로 오차 그래디언트를 전파할 때 하위층으로 진행될수록 그래디언트가 점점 작아지면 하위층의 연결 가중치가 잘 업데이트되지 않아 학습이 잘 안됨. 이를 **그래디언트 소실** 이라고 함.  
  
- 반대로 하위층으로 진행될수록 그래디언트가 점점 커지면 가중치들이 비정상적으로 갱신될 경우 알고리즘은 발산함. 이를 **그래디언트 폭주** 라고 함.(**순환 신경망에서 자주 일어난다고 함**)  
  
일반적으로 불한정한 그래디언트는 층마다 학습 속도가 달라질 수 있어서 학습을 어렵게 만듦.  

---
## 1.1 글로럿과 He초기화
글로럿과 벤지오는 논문에서 신경망의 신호가 적절하게 흐르기 위해서는 **각 층의 출력에 대한 분산이 입력에 대한 분산과 같아야 한다** 라고 주장함. 그리고 **역방향에서 층을 통과하기 전과 후의 그래디언트 분산이 동일해야 한다** 라고 함.  
글로럿과 벤지오는 각 층의 연결 가중치를 무작위로 초기화하여 이를 해결했다고 함.  
> 어떤 활성함수를 쓰냐에 따라 해당 층의 가중치들의 초기화방법이 달라지는 듯  
  
|초기화 전략|활성함수|  
|:--------|:-------|  
|글로럿(세이비어) 초기화   | 리니어, 하이퍼볼릭 탄젠트, 로지스틱, 소프트맥스|  
|He|ReLU 및 그 변종들|  
|르쿤| SELU|  
  
케라스는 기본적으로 균등분포의 글로럿 초기화를 사용한다고 함.

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
l1 = keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")
l2 = keras.layers.Dense(100, kernel_initializer="glorot_uniform")
l3 = keras.layers.Dense(100, kernel_initializer="lecun_normal")

**kernel_initializer** 매개변수를 조절하여 가중치 초기화를 조정할 수 있음

## 1.2 ReLU 활성 함수
글로럿과 벤지오의 2010년 논문에서 얻은 통찰 중 하나로 활성 함수를 잘못 선택하면 그래디언트 소실, 폭주 문제를 일으킬 수 있다고 한다.  
그 전에는 대부분 생물학적 뉴런의 방식과 비슷한 시그모이드 활성 함수가 최선이라고 생각했지만 실제론 다른 활성 함수가 훨씬 더 잘 작동한다는 사실이 밝혀졌음. 특히 **ReLU** 함수는 특정 양숫값에 수렴하지 않는다는 큰 장점이 있음.(그리고 계산도 빠름)  

하지만 안타깝게도 ReLU 함수도 완벽하지는 않음. **죽은 ReLU (dying ReLU)** 로 알려진 문제가 있는데 이는 학습하는 동안 일부 뉴런이 0 이외의 값을 출력하지 않는 현상을 말함.
> 특히 큰 학습률을 사용하면 신경망의 뉴런 절반이 죽어 있기도 한다고 함.   
>뉴런의 가중치가 바뀌어 학습 세트에 있는 모든 샘플에 대해 입력의 가중치 합이 음수가 되면 뉴런이 죽게 됨.  

가중치 합이 음수이면 ReLU함수의 그래디언트가 0이 되므로 경사 하강법이 더 작동하지 않기 때문.

---
이 문제를 해결하기 위해 **LeakyReLU** 같은 변종이 나타남. 이 함수의 하이퍼파라미터 **알파**로 z < 0 일 때 기울기를 정함.(일반적으로 0.01)  
이 작은 기울기가 LeakyReLU를 절대 죽지 않게 만들어 줌.  
> LeakyReLU가 ReLU보다 항상 성능이 높다는 실험 결과가 있다고 함
> 그리고 알파는 0.2(많이 통과)로 하는 것이 0.01(조금 통과)보다 더 나은 성능을 낸다고 함.



---
마지막으로 **ELU**함수가 있음.  
> 다른 모든 ReLU 변종 함수보다 성능이 높다고 함.  
학습 시간도 줄고 신경망의 테스트 셋 성능도 더 높았다고 함.

- z < 0 일 때 음숫값이 들어오므로 **활성 함수의 평균 출력이 0에 더 가까워짐**. 이는 그레디언트 소실 문제를 완화해줌. 하이퍼파라미터 **알파**는 z가 큰 음숫값일 때 수렴할 값을 정의함. (보통 1로 설정)  
- z < 0 이어도 그레디언트가 0이 아니므로 죽은 뉴런을 만들지 않음.  
- 알파 = 1 이면 이 함수는 z = 0 일 때 급격히 변동하지 않으므로 z = 0을 포함해 모든 구간에서 매끄러워 경사 하강법의 속도를 높여줌.

ELU함수의 주요 단점은 지수 함수를 사용하므로 ReLU나 그 변종들보다 계산이 느리다는 것임. 신기하게도 학습할 때는 수렴 속도가 빨라서 느린 계산이 상쇄된다고 함. 하지만 테스트 시에는 확실히 ReLU를 사용한 네트워크보다 느림.

---
스케일이 조정된 ELU 함수의 변종인 **SELU** 도 있음.  
개발 저자는 풀리 커넥티드 층만 쌓아서 신경망을 만들었을 때 모든 은닉층이 SELU함수를 사용한다면 네트워크가 **자기 정규화**된다는 것을 보였다고 함. 학습하는 동안 각 층의 출력이 평균 0, 표준 편차 1을 유지하는 경향이 있는 것임. 이는 그레디언트 폭주, 소실 문제를 막아줌.  
> 그 결과로 SELU함수는 이런 종류의 네트워크(아주 깊은 풀리 커넥티드 네트워크)에서 다른 활성 함수보다 뛰어난 성능을 냄.  

하지만 몇몇 조건이 있음.
- 입력 특성이 반드시 표준화 되어야 함.
- 모든 은닉층의 가중치는 르쿤 정규분포 초기화로 초기화해야 함. (kernel_initializer="lecun_normal")
- 네트워크는 일렬로 쌓은 층으로 구성되어야 함. 순환 신경망이나 스킵 커넥션과 같이 순차적이지 않은 구조에 SELU를 사용하면 자기 정규화를 보장하지 못한다고 함.

> 해당 논문은 네트워크의 모든 층이 풀리 커넥티드일 때만 자기 정규화를 보장함.  
하지만 일부 연구자들은 **합성곱 신경망**에서도 SELU함수가 성능을 향상시킬 수 있다고 말함

---
그렇다면 심층 신경망의 은닉층에는 어떤 활성 함수를 써야 하는가??  
일반적으로  
> - SELU
> - ELU
> - LeakyReLU (와 그 외 변종들)
> - ReLU
> - tanh
> - 로지스틱

순으로 하면 됨.  
네트워크가 자기 정규화되지 못하는 구조라면 ELU가 SELU보다 성능이 더 나을 수 있다고 함.  
실행 속도가 중요하다면 LeakyReLU를 선택할 수 있음.  
신경망이 과대적합되었다면 **RReLU**, 학습 세트가 아주 크다면 **PReLU**도 사용해볼 수 있음.  
ReLU가 가장 널리 사용되므로 많은 라이브러리와 하드웨어 가속기들이 ReLU에 특화되어 최적화되어있다고 함. 그래서 속도가 중요하면 ReLU가 가장 좋다고 함.

In [3]:
model = keras.models.Sequential([
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.PReLU()
])

LeakyReLU, PReLU, SELU를 사용하는 예시.  
RReLU는 케라스가 공식적으로 구현하지 않음.

---
## 1.3 배치 정규화
ELU(또는 다른 ReLU 변종)와 함께 He초기화를 사용하면 학습 초기 단계에서의 그레디언트 소실, 폭주를 크게 감소시킬 수 있지만 학습하는 동안 그 문제가 다시 발생하지 않으리란 보장은 없음.  
2015년의 한 논문에서 세르게이 이오페와 피리수티언 세게지가 그레디언트 소실, 폭주 문제를 해결하기 위해 **배치 정규화 (BN: batch normalization)** 기법을 제안함.  
>이 기법은 각 층에서 활성함수를 통과하기 전이나 후에 모델에 연산을 하나 추가함.  
>이 연산은 단순하게 입력을 원점에 맞추고 정규화한 다음 각 층에서 두 개의 새로운 파라미터로 결괏값의 스케일을 조정하고 이동시킴.  
>하나는 스케일 조정에, 다른 하나는 이동에 사용됨.

>많은 경우 **신경망의 첫 번째 층으로 배치 정규화를 추가하면**  학습 세트를 표준화할(StandardScaler와 같이) 필요가 없음.  
물론 한 번에 하나의 배치만 처리하기 때문에 근사적임. 또한 입력 특성마다 스케일을 조정하고 이동할 수 있음.

학습하는 동안 배치 정규화는 입력을 정규화한 다음 스케일을 조정하고 이동시킴. 하지만 테스트시에는 간단하지 않음. 샘플의 배치가 아니라 샘플 하나에 대한 예측을 만들어야 하기 때문.  
한 가지 방법은 학습이 끝난 후 전체 학습 세트를 신경망에 통과시켜 배치 정규화층의 각 입력에 대한 평균과 표준 편차를 계산하는 것. 예측할 때 배치 입력 평균과 표준 편차로 이 최종 입력 평균과 표준 편차를 대신 사용할 수 있음.  
> 그러나 대부분 배치 정규화 구현은 층의 입력 평균과 표준 편차의 이동 평균을 사용해 학습하는 동안 최종 통계를 추정함.  

케라스의 **BatchNormalization** 층은 이를 자동으로 수행함.  
정리하면 배치 정규화 층마다 네 개의 파라미터 벡터가 학습됨.  
- **감마** : 출력 스케일 벡터  
- **베타** : 출력 이동 벡터
- **뮤** : 최종 입력 평균 벡터
- **델타** : 최종 입력 표준 편차 벡터
감마와 베타는 일반적인 역전파를 통해 학습. 뮤와 델타는 지수 이동 평균을 사용하여 추정. 뮤와 델타는 학습하는 동안 추정되지만 학습이 끝난 후에 사용됨.(배치 입력 평균과 표준편차를 대신하기 위해)

---
#### 배치 정규화의 장점
>아오페와 세게디는 실험한 모든 심층 신경망에서 배치 정규화가 성능을 크게 향상시킨다는 것을 보임.  
특히 이미지넷 분류작업에서 큰 성과를 냄.

- 그래디언트 소실 문제가 크게 감소하여 tanh, 로지스틱 활성 함수같은 수렴성을 가진 활성 함수도 사용할 수 있음.
- 또 가중치 초기화에 네트워크가 훨씬 덜 민감해짐 
- 훨씬 큰 학습률을 사용하여 학습 과정의 속도를 크게 높일 수 있었음.
- 배치 정규화는 **규제 역할**을 하여 다른 규제 기법의 필요성을 줄여줌

---
#### 배치 정규화의 단점
- 모델의 복잡도를 키움.
- 층마다 추가되는 계산이 신경망의 예측을 느리게 하여 실행 시간 면에서도 손해 (그래도 학습이 끝나면 이전 층과 배치 정규화 층을 합쳐서 실행 속도 저하를 피할 수 있긴 함)

In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    
    keras.layers.Dense(10, activation="softmax")
])

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_5 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_6 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

케라스로 배치 정규화 구현하기.  
사실 은닉층 두 개를 가진 작은 예제라서 배치 정규화가 큰 도움이 되진 않을 듯.  
하지만 깊은 네트워크에선 엄청난 차이를 만들 수 있음.

In [6]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [7]:
model.layers[1].updates

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


[]

---
#### 배치 정규화층의 위치
배치 정규화 논문의 저자들은 활성함수 이후보다는 이전에 배치 정규화 층을 추가하는 것이 좋다고 조언함.  
하지만 작업에 따라 선호되는 방식이 달라서 이 조언에 대해서는 논란이 있음.  
> 즉 두 가지 방법 모두 실험해보고 어떤 것이 주어진 데이터셋에 가장 잘 맞는지 확인하는 것이 좋음.

활섬함수 이전에 배치 정규화 층을 추가하려면 은닉층에서 활성함수를 지정하지 않고 별도의 층으로 추가해야 함.  
그리고 배치 정규화 층은 입력마다 이동 파라미터를 포함하기 때문에 **이전 층에서 편향을 뺄 수 있음.** (use_bias = False) 

In [8]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    
    keras.layers.Dense(10, activation="softmax")
])

---
#### 배치 정규화의 하이퍼파라미터
보통 기본값이 잘 동작하지만 이따금 **momentum** 매개변수를 변경해야 할 수 있음.  
배치 정규화 층이 지수 이동 평균을 업데이트할 때 이 하이퍼파라미터를 사용함.  
> 적절한 모멘텀값은 일반적으로 1에 가까움. (0.9, 0.99, 0.999)  
데이터셋이 크고 미니배치가 작으면 소수점 뒤에 9를 더 넣어 1에 더 가깝게 조절

다른 중요한 하이퍼파라미터는 **axis**임.  
> 이 매개변수는 정규화할 축을 결정함.  
자신이 갖고있는 데이터셋에 맞춰 잘 설정해야 함.

기본적으로 **-1**로 설정되어 있음. 즉 마지막 축을 정규화함.
예를 들어 입력 배치가 [샘플 개수, 특성 개수]이면 입력 특성이 배치에 있는 모든 샘플에 대해 계산한 평균과 표준편차를 기반으로 정규화됨.  
입력 배치가 이미지라서 [샘플 개수, 높이, 너비]이면 배치 정규화층이 28개의 평균과 28개의 표준편차를 계산하여(**열에 있는 모든 행과 샘플에 대해 계산하므로 픽셀의 열마다 1개씩 만들어짐**) 동일한 평균과 표준편차를 사용하여 해당 열의 모든 픽셀을 정규화함.
>784개의 픽셀을 독립적으로 다루고 싶다면 **aixs = [1, 2]** 로 설정해야 함

---
## 1.4 그래디언트 클리핑
그래디언트 폭주 문제를 완화하는 인기 있는 다른 방법은 역전파될 때 일정 임곗값을 넘어서지 못하게 그래디언트를 잘라내는 것임.  
이를 **그래디언트 클리핑** 이라고 함.
> 순환 신경망같이 배치 정규화를 적용하기 어려운 경우에 이 방법을 많이 사용함.  
대부분의 경우는 배치 정규화로 충분하다고 함.

In [9]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)

이 옵티마이저는 그래디언트 벡터의 모든 원소를 -1.0 ~ 1.0 으로 클리핑함. 즉 학습되는 각 파라미터에 대한 손실의 모든 편미분 값을 잘라낸다는 것임. 임곗값은 하이퍼파라미터로 튜닝할 수 있음.  
> 클리핑 방법은 그래디언트의 방향을 바꿀 수 있음.

예를 들어 그레디언트 벡터가 [0.9, 100.0]이라면 그래디언트는 대체로 두 번째 축 방향을 향할 것임.  
하지만 클리핑하여서 그레디언트 벡터가 [0.9, 1.0]이라면 방향은 두 축의 중간을 향할 것임.  
실전에서 이 방식이 잘 동작한다고 함.  
만약 이 방향을 유지하고 싶다면 **clipnorm** 을 사용하면 됨.  
그러면 [0.009, 0.999] 이런 식으로 클리핑되어서 방향을 유지할 수 있음.
> 학습 중에 그레디언트가 폭주한다면 다른 임곗값으로 **값과 노름 모두 사용하여 클리핑할 수도 있음**.